1. Collect a Dataset
2. Preprocess the Dataset:
3. GAN Model Selection
>* DCGAN
>* StyleGAN.
3. Data Preparation
>* generator dataset (images to emulate)
>* discriminator dataset (images that are not part of the generator dataset).
4. GAN Training: Train the GAN model. 
5. Unique Image Generation
>* introduce random noise or variations into the input to the generator.
6. Evaluation
7. Adjust Hyperparameters

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array


<b>Load Data</b>

In [ ]:
# List of subfolders containing flower categories
subfolders = [
    # "./Data/flower_photos/daisy",
    #"./Data/flower_photos/dandelion",
    "./Data/flower_photos/roses",
    #"./Data/flower_photos/sunflowers",
    #"./Data/flower_photos/tulips"
]


> <b>Display samples</b> 

In [ ]:
# Display sample images from each subfolder
num_samples_per_class = 3
plt.figure(figsize=(12, 10))

for i, subfolder in enumerate(subfolders):
    class_name = os.path.basename(subfolder)
    image_files = os.listdir(subfolder)[:num_samples_per_class]

    for j, image_file in enumerate(image_files):
        img_path = os.path.join(subfolder, image_file)
        img = load_img(img_path, target_size=(64, 64))
        img_array = img_to_array(img) / 255.0 
        
        plt.subplot(len(subfolders), num_samples_per_class, i * num_samples_per_class + j + 1)
        plt.imshow(img_array)
        plt.title(class_name)
        plt.axis('off')

plt.tight_layout()
plt.show()


<h2>Preprocess Data</h2>

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

* <b>Resize</b>

In [ ]:
# Specify input and output folders
input_folder = './Data/flower_photos/daisy'
output_folder = './Data/daisy_resize_64'

# Specify target image size
target_size = (64, 64)

In [ ]:
# Function to resize
def resize_images(input_folder, output_folder, target_size):
    for filename in os.listdir(input_folder):
        image_path = os.path.join(input_folder, filename)
        
        # Check if the image file exists
        if not os.path.isfile(image_path):
            print(f"Skipping {filename}. File does not exist.")
            continue

        img = cv2.imread(image_path)
        
        # Check if the image is loaded successfully
        if img is None:
            print(f"Failed to load {filename}. Skipping.")
            continue
        
        # Resize and save the image
        try:
            img = cv2.resize(img, target_size)
            cv2.imwrite(os.path.join(output_folder, filename), img)
        except Exception as e:
            print(f"Error processing {filename}: {e}")

In [ ]:
# Call the function
resize_images(input_folder, output_folder, target_size)


* <b>Normalise</b>

In [ ]:
# Function to normalise
def normalize_images(input_folder, output_folder):
    for filename in os.listdir(input_folder):
        image_path = os.path.join(input_folder, filename)
        
        if not os.path.isfile(image_path):
            print(f"Skipping {filename}. File does not exist.")
            continue

        img = cv2.imread(image_path)
        
        if img is None:
            print(f"Failed to load {filename}. Skipping.")
            continue
        
        img = cv2.normalize(img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        cv2.imwrite(os.path.join(output_folder, filename), img * 255.0)


In [ ]:
# Specify input and output folders
input_folder = './Data/daisy_resize_64'
output_folder = './Data/daisy_normalise'

# Call the function
normalize_images(input_folder, output_folder)

* <b>Split Train-Test</b>

In [ ]:
input_folder = './Data/daisy_normalise'
output_folder = './Data/daisy_normalise'

In [ ]:
# Function to split data into train, validation, and test sets
def split_data(input_folder, output_folder, validation_split, test_split):
    all_files = os.listdir(input_folder)
    train_and_val_files, test_files = train_test_split(all_files, test_size=test_split)
    train_files, val_files = train_test_split(train_and_val_files, test_size=validation_split / (1 - test_split))

    os.makedirs(os.path.join(output_folder, 'train'), exist_ok=True)
    os.makedirs(os.path.join(output_folder, 'validation'), exist_ok=True)
    os.makedirs(os.path.join(output_folder, 'test'), exist_ok=True)

    for filename in train_files:
        os.rename(os.path.join(input_folder, filename), os.path.join(os.path.join(output_folder, 'train'), filename))
    for filename in val_files:
        os.rename(os.path.join(input_folder, filename), os.path.join(os.path.join(output_folder, 'validation'), filename))
    for filename in test_files:
        os.rename(os.path.join(input_folder, filename), os.path.join(os.path.join(output_folder, 'test'), filename))

# Perform data preprocessing
split_data(output_folder, output_folder, validation_split=0.1, test_split=0.1)


* <b>Data Augmentation</b>

<h2>Modeling</h2>

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Dense, Reshape, Flatten, AveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical

# Define constants
latent_dim = 512
image_size = 64

* <b>Generator</b>

In [ ]:
# Generator
def build_generator(latent_dim, image_size):
    input_noise = Input(shape=(latent_dim,))
    
    x = Dense(4 * 4 * 512)(input_noise)
    x = Reshape((4, 4, 512))(x)
    
    x = layers.UpSampling2D(size=(2, 2))(x)
    x = layers.Conv2D(256, kernel_size=3, padding="same")(x)
    x = layers.BatchNormalization(momentum=0.8)(x)
    x = layers.ReLU()(x)
    
    x = layers.UpSampling2D(size=(2, 2))(x)
    x = layers.Conv2D(128, kernel_size=3, padding="same")(x)
    x = layers.BatchNormalization(momentum=0.8)(x)
    x = layers.ReLU()(x)
    
    x = layers.UpSampling2D(size=(2, 2))(x)
    x = layers.Conv2D(64, kernel_size=3, padding="same")(x)
    x = layers.BatchNormalization(momentum=0.8)(x)
    x = layers.ReLU()(x)
    
    x = layers.UpSampling2D(size=(2, 2))(x)
    x = layers.Conv2D(3, kernel_size=3, padding="same")(x)  # Set the number of output channels to 3 for RGB images
    output_image = layers.Activation("tanh")(x)
    
    return Model(input_noise, output_image, name="tula_generator")


In [ ]:
# Build the generator
generator = build_generator(latent_dim, image_size)
generator.summary()

* <b>Discriminator</b>

In [ ]:
# Discriminator
def build_discriminator(image_size):
    input_image = Input(shape=(image_size, image_size, 3))
    
    x = layers.Conv2D(32, kernel_size=3, strides=2, padding="same")(input_image)
    x = layers.LeakyReLU(alpha=0.2)(x)
    
    x = layers.Conv2D(64, kernel_size=3, strides=2, padding="same")(x)
    x = layers.BatchNormalization(momentum=0.8)(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    
    x = layers.Conv2D(128, kernel_size=3, strides=2, padding="same")(x)
    x = layers.BatchNormalization(momentum=0.8)(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    
    x = layers.Conv2D(256, kernel_size=3, strides=2, padding="same")(x)
    x = layers.BatchNormalization(momentum=0.8)(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    
    x = layers.Flatten()(x)
    validity = layers.Dense(1, activation="sigmoid")(x)
    
    return Model(input_image, validity, name="tula_discriminator")

# discriminator.summary()

<h3>Model Training</h3>

In [ ]:
image_height = 64
image_width = 64
num_channels = 3 

In [ ]:
# Define your GAN architecture (generator and discriminator)
latent_dim = 100
#image_size = (64, 64, 3) 

generator = build_generator(latent_dim, (image_height, image_width, num_channels))
discriminator = build_discriminator(image_size)

# Compile the discriminator
discriminator.compile(loss='binary_crossentropy',
                      optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5),
                      metrics=['accuracy'])

# Create the GAN by chaining the generator and discriminator
discriminator.trainable = False  # Freeze the discriminator when training the GAN
gan_input = tf.keras.layers.Input(shape=(latent_dim,))
x = generator(gan_input)
gan_output = discriminator(x)
gan = tf.keras.models.Model(gan_input, gan_output)

# Compile the GAN
gan.compile(loss='binary_crossentropy',
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5))


In [ ]:
# Training parameters
batch_size = 64
epochs = 10000
sample_interval = 200  

# Training loop
for epoch in range(epochs):
    # Train the discriminator
    idx = np.random.randint(0, train_images.shape[0], batch_size)
    real_images = train_images[idx]

    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    generated_images = generator.predict(noise)

    real_labels = np.ones((batch_size, 1))
    fake_labels = np.zeros((batch_size, 1))

    d_loss_real = discriminator.train_on_batch(real_images, real_labels)
    d_loss_fake = discriminator.train_on_batch(generated_images, fake_labels)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train the generator
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    valid_labels = np.ones((batch_size, 1))
    g_loss = gan.train_on_batch(noise, valid_labels)

    # Print progress and save generated images at specified intervals
    if epoch % sample_interval == 0:
        print(f"Epoch {epoch}, D Loss: {d_loss[0]}, G Loss: {g_loss}")
        # Generate and save sample images
        save_generated_images(epoch, generator, save_dir="generated_images")

In [ ]:

# After training, you can generate images using the trained generator:
def generate_images(generator, latent_dim, num_images, save_dir):
    noise = np.random.normal(0, 1, (num_images, latent_dim))
    generated_images = generator.predict(noise)

    for i in range(num_images):
        image = generated_images[i]
        image = (image + 1) * 127.5  # De-normalize
        image = image.astype(np.uint8)
        save_path = os.path.join(save_dir, f"generated_image_{i}.png")
        cv2.imwrite(save_path, image)

generate_images(generator, latent_dim, num_images=10, save_dir="generated_images")


* <b>Tula Archetecture</b>

In [ ]:
tula_model.summary()

In [ ]:
# Generate random noise (replace this with your actual noise data)
batch_size = 16  
latent_dim = 100 
noise = np.random.normal(0, 1, (batch_size, latent_dim))

# Generate images using the generator
generated_images = generator.predict(noise)

In [ ]:
# Create a grid to display multiple images
rows = 4 
cols = 4 

# Set up the figure and axes
fig, axs = plt.subplots(rows, cols)
fig.subplots_adjust(hspace=0.5)

# Display the generated images in the grid
for i in range(rows):
    for j in range(cols):
        index = i * cols + j
        if index < batch_size:
            axs[i, j].imshow(generated_images[index])
            axs[i, j].axis("off")

# Show the plot
plt.show()